In [ ]:
#| default_exp mingpt
%load_ext autoreload
%autoreload 2

In [ ]:
do_download:bool=False
if do_download:
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# | export
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# | export
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device

In [ ]:
is_dylan: bool = False
if is_dylan:
    df = pd.read_csv("./bob_dylan_lyrics.csv")
    text = "\n\n".join(df.lyrics.values.tolist())
else:
    with open("./input.txt", "r", encoding="utf-8") as f:
        text = f.read()

    len(text), text[:100]
text

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}


def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]


def decode(l: list[int]) -> str:
    return "".join([itos[i] for i in l])


decode(encode("hello world"))
encode("hello world")

In [ ]:
# Compute train and validation data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(text))
train = data[:n]
val = data[n:]

In [ ]:
# hyperparams
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2

eval_iters = 200

In [ ]:
# every time we set the seed it reinitialize the count of random generation, which is used
torch.manual_seed(42)


def get_batch(split: str):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # ix are random starting points of each batch
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y

In [ ]:
b = get_batch("train")

In [ ]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_emb = torch.nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_emb(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [ ]:
model = BigramModel(vocab_size)
model = model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


train_losses = []
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        train_losses.append(losses["train"].item())
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

In [ ]:
plt.plot(np.array(train_losses))

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export("mingpt.ipynb")